In [21]:
import ipywidgets as widgets
from IPython.display import display
import datetime
import csv
import pandas as pd

CSV_FILE_PATH = "./data.csv"

class SocialRow:
    def __init__(self, created_date: datetime.date, twitter: int, instagram: int, tiktok: int) -> None:
        self.created_date = created_date
        self.twitter = twitter
        self.instagram = instagram
        self.tiktok = tiktok

    def to_dict(self) -> dict:
        return {
            'Created Date': self.created_date,
            'Twitter': self.twitter,
            'Instagram': self.instagram,
            'TikTok': self.tiktok
        }
    
    @classmethod
    def get_header(cls) -> list:
        return ['Created Date', 'Twitter', 'Instagram', 'TikTok']

class CsvHandler:
    def __init__(self, file_path: str, header: list) -> None:
        self.file_path = file_path
        self.header = header

        self._add_header_if_not_exist() 
        pass
    
    def _add_header_if_not_exist(self) -> None:
        with open(self.file_path, 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=self.header)
            if csvfile.tell() == 0:
                writer.writeheader()

    
    
    # check if the row is valid related header
    def _check_row(self, row: dict) -> bool:
        return set(row.keys()) == set(self.header)
    
    
    # add row to csv file
    def add_row(self, row: dict) -> None:
        if self._check_row(row):
            with open(self.file_path, 'a') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=self.header)
                writer.writerow(row)

    
    def read(self) -> list:
        data = []
        with open(self.file_path, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                data.append(row)
        return data

    def get_header(self) -> list:
        return self.header
class WidgetManager:
    def __init__(self, csv_handler: CsvHandler) -> None:
        self.csv_handler = csv_handler
        self._create_widgets()
        pass

    def _create_widgets(self) -> None:
        self.twitter = widgets.IntText(description='twitter : ')
        self.instagram = widgets.IntText(description='instagram : ')
        self.tiktok = widgets.IntText(description='tiktok : ')
        # get the date value of the widgets
        self.datePicker = widgets.DatePicker()
        self.datePicker.value = datetime.date.today()
        
        self.button = widgets.Button(description="Write CSV")
        self.output = widgets.Output(layout={'border': '1px solid black'})
        
        self.my_widgets = widgets.VBox([self.datePicker, self.twitter,self.instagram, self.tiktok, self.button, self.output])
        
        self.button.on_click(self._on_click_button)
    
    def _update_data_grid(self) -> None:
        data = self.csv_handler.read()
        headers = self.csv_handler.get_header()
        df = pd.DataFrame(data[1:], columns=headers)
        display(df)

    def _write_csv(self) -> None:
        row = SocialRow(self.datePicker.value, self.twitter.value, self.instagram.value, self.tiktok.value).to_dict()
        self.csv_handler.add_row(row)

    def _get_widgets_values(self) -> dict:
        return {
            'Created Date': self.datePicker.value,
            'Twitter': self.twitter.value,
            'Instagram': self.instagram.value,
            'TikTok': self.tiktok.value
        }
    def _get_widgets_values_as_string(self) -> str:
        values = self._get_widgets_values()
        return f'Created Date : {values["Created Date"]}, Twitter : {values["Twitter"]}, Instagram : {values["Instagram"]}, TikTok : {values["TikTok"]}'
    
    def _on_click_button(self, button: widgets.Button) -> None:
        values = self._get_widgets_values_as_string()
        print(f'write values : {values}')
        self._write_csv()
        self._update_data_grid()
        self.output.capture(clear_output=True, wait=True)
        
    def show_widgets(self) -> None:
        self._update_data_grid()
        display(self.my_widgets)


def main() -> None:
    # header = SocialRow.get_header()
    # print('header : ', header)
    
    csv_handler = CsvHandler(CSV_FILE_PATH, SocialRow.get_header())
    wm = WidgetManager(csv_handler)
    wm.show_widgets()
    
    
    


main()

,Created Date,Twitter,Instagram,TikTok
0,2023-04-18,26,0,52
